In [225]:
with open("input.txt", 'r') as file:
    inputs = file.read().split("\n\n")
    seeds, *inputs = [i.split(':') for i in inputs]
    seeds = [seed for seed in seeds[1].split(' ') if seed != '']
    input_map = {key: [s.split() for s in value.splitlines() if s != ''] for key, value in inputs}

with open("sample_input.txt", 'r') as file:
    inputs = file.read().split("\n\n")
    sample_seeds, *inputs = [i.split(':') for i in inputs]
    sample_seeds = [seed for seed in sample_seeds[1].split(' ') if seed != '']
    sample_input_map = {key: [s.split() for s in value.splitlines() if s != ''] for key, value in inputs}

In [226]:
def GetMapValue(value, inputMap):
    for s in inputMap:
        destination_range_start = int(s[0])
        source_range_start = int(s[1])
        range_length = int(s[2])

        if source_range_start <= value < source_range_start + range_length:
            return destination_range_start + value - source_range_start
    
    return value

def GetSeedValue(value, inputMap):
    for key in input_map.keys():
        value = GetMapValue(value, inputMap[key])
    return value

In [227]:
print(GetSeedValue(79, sample_input_map))
print(GetSeedValue(14, sample_input_map))
print(GetSeedValue(55, sample_input_map))
print(GetSeedValue(13, sample_input_map))

82
43
86
35


In [228]:
min([GetSeedValue(int(seed), input_map) for seed in seeds])

240320250

In [229]:
def GetSeedPairs(seeds):
    seedPairs = [[int(seeds[2*i]), int(seeds[2*i + 1])] for i in range(int(len(seeds) / 2))]
    ##return [s2[0] + i for s2 in seedPairs for i in range(s2[1])]
    return seedPairs

GetSeedPairs(seeds)

#min([GetSeedValue(int(seed), input_map) for seed in seeds])

[[3429320627, 235304036],
 [1147330745, 114559245],
 [1684000747, 468955901],
 [677937579, 96599505],
 [1436970021, 26560102],
 [3886049334, 159534901],
 [936845926, 25265009],
 [3247146679, 95841652],
 [3696363517, 45808572],
 [2319065313, 125950148]]

In [230]:
def printMap(myMap):
    return [str(s[1]) + '-' + str(s[1] + s[2] - 1) + ' -> ' + str(s[0]) + '-' + str(s[0] + s[2] - 1) for s in myMap]

In [231]:
print(printMap(clean_map(sample_input_map['temperature-to-humidity map'])))
print(printMap(clean_map(sample_input_map['humidity-to-location map'])))

['0-68 -> 1-69', '69-69 -> 0-0', '70-70 -> 70-70']
['0-55 -> 0-55', '56-92 -> 60-96', '93-96 -> 56-59', '97-97 -> 97-97']


In [266]:
def clean_map(my_map):
    values = [int(entry[1]) for entry in my_map]
    if min(values) != 0:
        my_map.append(['0', '0', str(min(values))])

    max_index = values.index(max(values))
    max_entry = my_map[max_index]
    if my_map[max_index][2] != '1' or my_map[max_index][0] != my_map[max_index][1]:
        my_map.append([str(int(max_entry[1]) + int(max_entry[2])), str(int(max_entry[1]) + int(max_entry[2])), str(1)])

    values = [int(entry[1]) for entry in my_map]
    sorted_indexes = sorted(range(len(values)),key=values.__getitem__)
    my_map = [my_map[i] for i in sorted_indexes]
    my_map = [[int(m[0]), int(m[1]), int(m[2])] for m in my_map]
    return my_map

def mergeMaps(inputMap, outputMap):
    maxValue = max(max([element[1] for element in inputMap]), max([element[1] for element in outputMap]))

    newMap = []
    input_i = 0
    curInput = inputMap[input_i][1]
    curRange = 0
    while curInput < maxValue:
        curInput += curRange

        inputValue = GetMapValue(curInput, inputMap)
        outputValue = GetMapValue(inputValue, outputMap)

        inputRange = inputMap[input_i][1] + inputMap[input_i][2] - curInput

        output_i = 0
        while outputMap[output_i][1] + outputMap[output_i][2] <= inputValue:
            output_i += 1
            if output_i == len(outputMap) - 1:
                break

        outputRange = outputMap[output_i][1] + outputMap[output_i][2] - inputValue

        print("Current Input: " + str(curInput) + "\n" + "Input:  Index " + str(input_i) + " Value " + str(inputValue) + " Range " + str(inputRange) + "\nOutput: Index " + str(output_i) + " Value " + str(outputValue) + " Range " + str(outputRange) +"\n")

        if output_i == len(outputMap) - 1 or (input_i != len(inputMap) - 1 and inputRange < outputRange and inputRange > 0):
            curRange = inputRange
            input_i += 1
        else:
            curRange = outputRange

        newMap.append([outputValue, curInput, curRange])
    
    return clean_map(newMap)


def MergeAllMaps():
    keys = list(input_map.keys())

    curMap = clean_map(sample_input_map[keys[0]])

    for i in range(1, len(keys)):
        curMap = mergeMaps(curMap, clean_map(sample_input_map[keys[i]]))
    return curMap

In [267]:
inputMap = clean_map(sample_input_map['temperature-to-humidity map'])
outputMap = clean_map(sample_input_map['humidity-to-location map'])

mergeMaps(inputMap, outputMap)

Current Input: 0
Input:  Index 0 Value 1 Range 69
Output: Index 0 Value 1 Range 55

Current Input: 55
Input:  Index 0 Value 56 Range 14
Output: Index 1 Value 60 Range 37

Current Input: 69
Input:  Index 1 Value 0 Range 1
Output: Index 0 Value 0 Range 56

Current Input: 70
Input:  Index 2 Value 70 Range 1
Output: Index 1 Value 74 Range 23

Current Input: 93
Input:  Index 2 Value 93 Range -22
Output: Index 2 Value 56 Range 4

Current Input: 97
Input:  Index 2 Value 97 Range -26
Output: Index 3 Value 97 Range 1



[[1, 0, 55],
 [60, 55, 14],
 [0, 69, 1],
 [74, 70, 23],
 [71, 71, 1],
 [56, 93, 4],
 [97, 97, -26]]

In [261]:
print(printMap(inputMap))
print(printMap(outputMap))

['0-68 -> 1-69', '69-69 -> 0-0', '70-70 -> 70-70']
['0-55 -> 0-55', '56-92 -> 60-96', '93-96 -> 56-59', '97-97 -> 97-97']


In [241]:
mergedMaps = MergeAllMaps()

In [244]:
GetMapValue(79, mergedMaps)

78

In [249]:
test = mergeMaps(clean_map(sample_input_map['seed-to-soil map']), clean_map(sample_input_map['soil-to-fertilizer map']))

In [251]:
print(GetMapValue(79, test))
print(GetMapValue(14, test))
print(GetMapValue(55, test))
print(GetMapValue(13, test))

81
53
57
52


In [252]:
test

[[39, 0, 15],
 [0, 15, 35],
 [37, 50, 2],
 [54, 52, 46],
 [35, 98, 2],
 [100, 100, 1],
 [101, 101, 1]]

In [219]:
myMap = mergeMaps(clean_map(sample_input_map[keys[0]]), clean_map(sample_input_map[keys[1]]))
#myMap = mergeMaps(myMap, clean_map(sample_input_map[keys[3]]))
#myMap = mergeMaps(myMap, clean_map(sample_input_map[keys[4]]))

100
Current Input: 0
Input: Index 0 Value 0 Range 50
Output: Index 0 Value 39 Range 15

Current Input: 15
Input: Index 0 Value 15 Range 35
Output: Index 1 Value 0 Range 37

Current Input: 50
Input: Index 1 Value 52 Range 48
Output: Index 2 Value 37 Range 2

Current Input: 52
Input: Index 1 Value 54 Range 46
Output: Index 3 Value 54 Range 1

Current Input: 98
Input: Index 2 Value 50 Range 2
Output: Index 2 Value 35 Range 4

Current Input: 100
Input: Index 3 Value 100 Range 1
Output: Index 3 Value 100 Range -45



In [220]:
brokenMap = mergeMaps(myMap, clean_map(sample_input_map[keys[2]]))

101
Current Input: 0
Input: Index 0 Value 39 Range 15
Output: Index 3 Value 28 Range 22

Current Input: 15
Input: Index 1 Value 0 Range 35
Output: Index 0 Value 42 Range 7

Current Input: 22
Input: Index 1 Value 7 Range 28
Output: Index 1 Value 57 Range 4

Current Input: 26
Input: Index 1 Value 11 Range 24
Output: Index 2 Value 0 Range 42

Current Input: 50
Input: Index 2 Value 37 Range 2
Output: Index 3 Value 26 Range 24

Current Input: 52
Input: Index 3 Value 54 Range 46
Output: Index 4 Value 50 Range 8

Current Input: 98
Input: Index 4 Value 35 Range 2
Output: Index 3 Value 24 Range 26

Current Input: 100
Input: Index 5 Value 100 Range 1
Output: Index 4 Value 100 Range -38

Current Input: 101
Input: Index 6 Value 101 Range 1
Output: Index 4 Value 101 Range -39



In [221]:
print(printMap(myMap))
print(printMap(clean_map(sample_input_map[keys[2]])))
print(brokenMap)

['0-14 -> 39-53', '15-49 -> 0-34', '50-51 -> 37-38', '52-97 -> 54-99', '98-99 -> 35-36', '100-100 -> 100-100', '101-101 -> 101-101']
['0-6 -> 42-48', '7-10 -> 57-60', '11-52 -> 0-41', '53-60 -> 49-56', '61-61 -> 61-61']
[[28, 0, 15], [42, 15, 7], [57, 22, 4], [0, 26, 24], [26, 50, 2], [50, 52, 46], [24, 98, 2], [100, 100, 1], [101, 101, 1], [102, 102, 1]]
